# Tic Tac Toe (Minimax + GUI)
Brief: Implements tic tac toe logic with minimax and a Tkinter interface.
Run: execute the logic cell, then the GUI cell, then call `start_ui()` (Cell 3) to open the play window.
Play: choose X or O, click New Game, make moves; AI replies optimally.
Notes: uses only the standard library; the Tkinter window opens outside the notebook.

## Core game logic
Implements board representation, turn logic, win/draw detection, and the minimax solver used by the UI.

In [ ]:
import copy

X = "X"
O = "O"
EMPTY = None


def initial_state():
    """Return a fresh empty 3x3 board with None placeholders."""
    return [[EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY]]


def player(board):
    """Return the player whose turn it is (X starts, then alternate)."""
    count = 0
    for i in range(3):
        for j in range(3):
            if board[i][j] == X:
                count += 1
            if board[i][j] == O:
                count -= 1

    return X if count < 1 else O


def actions(board):
    """Return a set of available (row, col) moves for the given board."""
    possible_moves = set()
    for i in range(3):
        for j in range(3):
            if board[i][j] is EMPTY:
                possible_moves.add((i, j))

    return possible_moves if len(possible_moves) != 0 else {}


def result(board, action):
    """Return the board that results from making move (row, col) for the current player."""
    if board[action[0]][action[1]] != EMPTY:
        raise Exception("Error, not valid action")

    temp = copy.deepcopy(board)
    temp[action[0]][action[1]] = player(board)
    return temp


def winner(board):
    """Return X or O if a player has three in a row, else None."""
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != EMPTY:
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != EMPTY:
            return board[0][i]

    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return board[0][2]

    return None


def terminal(board):
    """Return True if the game is over (win or draw), otherwise False."""
    if winner(board) != None:
        return True
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                return False
    return True


def utility(board):
    """Return 1 if X has won, -1 if O has won, 0 otherwise."""
    w = winner(board)
    return -1 if w is O else 1 if w is X else 0


def min_value(board):
    """Minimax value assuming the minimizing player (O) acts optimally."""
    if terminal(board):
        return utility(board)

    best_move = float("inf")
    for action in actions(board):
        best_move = min(best_move, max_value(result(board, action)))

    return best_move


def max_value(board):
    """Minimax value assuming the maximizing player (X) acts optimally."""
    if terminal(board):
        return utility(board)

    best_move = float("-inf")
    for action in actions(board):
        best_move = max(best_move, min_value(result(board, action)))

    return best_move


def minimax(board):
    """Return the optimal action for the current player, or None if terminal."""
    if terminal(board):
        return None

    if player(board) is X:
        best_score = float("-inf")
        best_move = None
        for action in actions(board):
            val = max_value(result(board, action))
            if val > best_score:
                best_score = val
                best_move = action
        return best_move

    elif player(board) is O:
        best_score = float("inf")
        best_move = None
        for action in actions(board):
            val = min_value(result(board, action))
            if val < best_score:
                best_score = val
                best_move = action
        return best_move

## GUI layer
Builds the Tkinter window, binds buttons to moves, and wires the minimax AI responses.

In [ ]:
import tkinter as tk
from tkinter import messagebox


class TicTacToeGUI:
    """Tkinter-based interface that plays against the minimax AI."""

    def __init__(self):
        """Build the window, controls, and initialize board state."""
        self.root = tk.Tk()
        self.root.title("Tic Tac Toe - Minimax")
        self.board = initial_state()
        self.human = X
        self.buttons = []

        controls = tk.Frame(self.root, padx=10, pady=10)
        controls.pack(side=tk.TOP, fill=tk.X)

        tk.Label(controls, text="Choose your marker:").pack(side=tk.LEFT)
        self.side_var = tk.StringVar(value=self.human)
        tk.Radiobutton(
            controls, text="Play as X", variable=self.side_var, value=X
        ).pack(side=tk.LEFT)
        tk.Radiobutton(
            controls, text="Play as O", variable=self.side_var, value=O
        ).pack(side=tk.LEFT)
        tk.Button(controls, text="New Game", command=self.new_game).pack(
            side=tk.LEFT, padx=(10, 0)
        )

        self.status_var = tk.StringVar(value="Select side and start playing")
        tk.Label(self.root, textvariable=self.status_var, pady=6).pack(side=tk.TOP)

        board_frame = tk.Frame(self.root, padx=10, pady=10)
        board_frame.pack()
        for r in range(3):
            row_buttons = []
            for c in range(3):
                btn = tk.Button(
                    board_frame,
                    text="",
                    width=6,
                    height=3,
                    font=("Helvetica", 18, "bold"),
                    command=lambda r=r, c=c: self.handle_click(r, c),
                )
                btn.grid(row=r, column=c, padx=4, pady=4)
                row_buttons.append(btn)
            self.buttons.append(row_buttons)

        self.refresh_ui()

    def new_game(self):
        """Reset the board using the chosen side and let AI start if needed."""
        self.human = self.side_var.get()
        self.board = initial_state()
        self.refresh_ui()
        if player(self.board) != self.human:
            self.ai_move()  # Let AI start if human picked O

    def handle_click(self, row, col):
        """Apply a human move, then schedule the AI response."""
        if terminal(self.board):
            return
        if player(self.board) != self.human:
            return  # Ignore clicks while AI thinking/turn
        if self.board[row][col] is not EMPTY:
            return
        self.board = result(self.board, (row, col))
        self.refresh_ui()
        if not terminal(self.board):
            self.root.after(150, self.ai_move)

    def ai_move(self):
        """Compute and play the AI's optimal move."""
        if terminal(self.board):
            return
        if player(self.board) == self.human:
            return
        action = minimax(self.board)
        if action is None:
            return
        self.board = result(self.board, action)
        self.refresh_ui()

    def refresh_ui(self):
        """Sync button labels/states with the board and show end-game dialogs."""
        for r in range(3):
            for c in range(3):
                val = self.board[r][c]
                self.buttons[r][c]["text"] = "" if val is None else val
                disabled = terminal(self.board) or player(self.board) != self.human
                self.buttons[r][c]["state"] = (
                    tk.DISABLED if disabled or val is not None else tk.NORMAL
                )
        self.status_var.set(self.status_message())

        if terminal(self.board):
            winner_mark = winner(self.board)
            if winner_mark:
                messagebox.showinfo("Game Over", f"{winner_mark} wins!")
            else:
                messagebox.showinfo("Game Over", "It's a draw!")

    def status_message(self):
        """Return a short status string describing whose turn it is."""
        if terminal(self.board):
            w = winner(self.board)
            return f"{w} wins!" if w else "Draw"
        turn = player(self.board)
        role = "You" if turn == self.human else "AI"
        return f"Turn: {role} ({turn})"

    def run(self):
        """Enter the Tkinter event loop."""
        self.root.mainloop()


def start_ui():
    """Launch the Tkinter window for interactive play."""
    app = TicTacToeGUI()
    app.run()

## Launch cell
Call `start_ui()` to open the play window.

In [ ]:
# Run start_ui() to launch the interface in a new window.
start_ui()